In [ ]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1OKFSv2GpuUFDphO0r8LdM7bl6MAWwBfX' -O data.csv

### Описание датасета

Короткое описание данных:
```
price: sale price (this is the target variable)
id: transaction id
timestamp: date of transaction
full_sq: total area in square meters, including loggias, balconies and other non-residential areas
life_sq: living area in square meters, excluding loggias, balconies and other non-residential areas
floor: for apartments, floor of the building
max_floor: number of floors in the building
material: wall material
build_year: year built
num_room: number of living rooms
kitch_sq: kitchen area
state: apartment condition
product_type: owner-occupier purchase or investment
sub_area: name of the district

The dataset also includes a collection of features about each property's surrounding neighbourhood, and some features that are constant across each sub area (known as a Raion). Most of the feature names are self explanatory, with the following notes. See below for a complete list.

full_all: subarea population
male_f, female_f: subarea population by gender
young_*: population younger than working age
work_*: working-age population
ekder_*: retirement-age population
n_m_{all|male|female}: population between n and m years old
build_count_*: buildings in the subarea by construction type or year
x_count_500: the number of x within 500m of the property
x_part_500: the share of x within 500m of the property
_sqm_: square meters
cafe_count_d_price_p: number of cafes within d meters of the property that have an average bill under p RUB
trc_: shopping malls
prom_: industrial zones
green_: green zones
metro_: subway
_avto_: distances by car
mkad_: Moscow Circle Auto Road
ttk_: Third Transport Ring
sadovoe_: Garden Ring
bulvar_ring_: Boulevard Ring
kremlin_: City center
zd_vokzaly_: Train station
oil_chemistry_: Dirty industry
ts_: Power plant
```

### Setup

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("train.csv", parse_dates=["timestamp"])

Разделите имеющиеся у вас данные на обучающую и тестовую выборки. В качестве обучающей выборки возьмите первые 80% данных, последние 20% - тестовая выборка.

In [ ]:
drop_columns = [
    'id',           # May leak information  # May leak information
]
cat_columns = [
    'product_type',              #
    'material',                  # Material of the wall
    'state',                     # Satisfaction level
    'sub_area',                  # District name
    'culture_objects_top_25',    #
    'thermal_power_plant_raion', #
    'incineration_raion',        #
    'oil_chemistry_raion',       #
    'radiation_raion',           #
    'railroad_terminal_raion',   #
    'big_market_raion',          #
    'nuclear_reactor_raion',     #
    'detention_facility_raion',  #
    'ID_metro',                  #
    'ID_railroad_station_walk',  #
    'ID_railroad_station_avto',  #
    'water_1line',               #
    'ID_big_road1',              #
    'big_road1_1line',           #
    'ID_big_road2',              #
    'railroad_1line',            #
    'ID_railroad_terminal',      #
    'ID_bus_terminal',           #
    'ecology',                   #
]
num_columns = list(set(df.columns).difference(set(cat_columns + drop_columns + ['price'])))

In [ ]:
from sklearn.impute import SimpleImputer
import numpy as np

In [ ]:
def clear(df):
    new = df.drop('id', axis=1)
    new['Investment'] = (new['product_type'] == 'Investment').astype(int)
    imputer = SimpleImputer(strategy='median')
    new['material'] = pd.Series(imputer.fit_transform(np.array(df['material']).reshape(-1, 1)).reshape(-1))
    new['state'] = pd.Series(imputer.fit_transform(np.array(df['state']).reshape(-1, 1)).reshape(-1))
    new = new.drop('sub_area', axis=1)
    new['culture_objects_top_25'] = (new['culture_objects_top_25'] == 'yes').astype(int)
    new['thermal_power_plant_raion'] = (new['thermal_power_plant_raion'] == 'yes').astype(int)
    new['incineration_raion'] = (new['incineration_raion'] == 'yes').astype(int)
    new['oil_chemistry_raion'] = (new['oil_chemistry_raion'] == 'yes').astype(int)
    new['radiation_raion'] = (new['radiation_raion'] == 'yes').astype(int)
    new['railroad_terminal_raion'] = (new['railroad_terminal_raion'] == 'yes').astype(int)
    new['big_market_raion'] = (new['big_market_raion'] == 'yes').astype(int)
    new['nuclear_reactor_raion'] = (new['nuclear_reactor_raion'] == 'yes').astype(int)
    new['detention_facility_raion'] = (new['detention_facility_raion'] == 'yes').astype(int)
    new['water_1line'] = (new['water_1line'] == 'yes').astype(int)
    new['big_road1_1line'] = (new['big_road1_1line'] == 'yes').astype(int)
    new['railroad_1line'] = (new['railroad_1line'] == 'yes').astype(int)
    new = new.drop(['product_type', 'ID_railroad_station_walk'], axis=1)
    '''new = new.drop(['product_type', 'ID_metro', 'ID_railroad_station_walk',
    'ID_railroad_station_avto', 'ID_big_road1', 'ID_big_road2',
    'ID_railroad_terminal', 'ID_bus_terminal'], axis=1)'''
    new = pd.concat([new, pd.get_dummies(df['ecology'])], axis=1).drop('ecology', axis=1)
    for column in num_columns:
        imputer = SimpleImputer(strategy='median')
        new[column] = pd.Series(imputer.fit_transform(np.array(new[column]).reshape(-1, 1)).reshape(-1))
    return new

In [ ]:
my_df = clear(df)

In [ ]:
from sklearn.model_selection import train_test_split
X = my_df.drop('price', axis=1)
y = my_df['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

Возможно в ваших моделях вам придется указывать, какие колонки являются категориальными (например, в бустингах). Для упрощения предлагается разделить колонки по следующему принципу:
```
drop_columns = [
    'id',           # May leak information
    'timestamp',    # May leak information
]
cat_columns = [
    'product_type',              #
    'material',                  # Material of the wall
    'state',                     # Satisfaction level
    'sub_area',                  # District name
    'culture_objects_top_25',    #
    'thermal_power_plant_raion', #
    'incineration_raion',        #
    'oil_chemistry_raion',       #
    'radiation_raion',           #
    'railroad_terminal_raion',   #
    'big_market_raion',          #
    'nuclear_reactor_raion',     #
    'detention_facility_raion',  #
    'ID_metro',                  #
    'ID_railroad_station_walk',  #
    'ID_railroad_station_avto',  #
    'water_1line',               #
    'ID_big_road1',              #
    'big_road1_1line',           #
    'ID_big_road2',              #
    'railroad_1line',            #
    'ID_railroad_terminal',      #
    'ID_bus_terminal',           #
    'ecology',                   #
]
num_columns = list(set(df.columns).difference(set(cat_columns + drop_columns)))
```

### Baseline

В качестве Baseline обучим `DecisionTreeRegressor` из `sklearn`.

In [ ]:
from sklearn.tree import DecisionTreeRegressor

In [ ]:
model = DecisionTreeRegressor()
model.fit(X_train, y_train)
predict = model.predict(X_test)

Проверим качество на отложенной выборке.

In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, predict, squared=False)

### Feature Engineering

Часто улучшить модель можно с помощью аккуратного Feature Engineering.

Добавим в модель дополнительные признаки:
* "Как часто в этот год и этот месяц появлились объявления"
* "Как часто в этот год и эту неделю появлялись объявления"

In [ ]:
month_year = (df.timestamp.dt.month + df.timestamp.dt.year * 100)
month_year_cnt_map = month_year.value_counts().to_dict()
df["month_year_cnt"] = month_year.map(month_year_cnt_map)

week_year = (df.timestamp.dt.weekofyear + df.timestamp.dt.year * 100)
week_year_cnt_map = week_year.value_counts().to_dict()
df["week_year_cnt"] = week_year.map(week_year_cnt_map)

Добавим следюущие дополнительные признаки:
* Месяц (из колонки `timestamp`)
* День недели (из колонки `timestamp`)
* Отношение "этаж / максимальный этаж в здании" (колонки `floor` и `max_floor`)
* Отношение "площадь кухни / площадь квартиры" (колонки `kitchen_sq` и `full_sq`)

In [ ]:
import datetime

In [ ]:
def new_features(df):
    new = clear(df)
    month_year = (df.timestamp.dt.month + df.timestamp.dt.year * 100)
    month_year_cnt_map = month_year.value_counts().to_dict()
    new["month_year_cnt"] = month_year.map(month_year_cnt_map)

    week_year = (df.timestamp.dt.weekofyear + df.timestamp.dt.year * 100)
    week_year_cnt_map = week_year.value_counts().to_dict()
    new["week_year_cnt"] = week_year.map(week_year_cnt_map)

    new['mounth'] = df['timestamp'].dt.month
    new['weekday'] = df['timestamp'].apply(lambda x: x.weekday())
    new['max_floor'] = new['max_floor'].apply(lambda x: -1 if x == 0 else x)
    new['full_sq'] = new['full_sq'].apply(lambda x: -1 if x == 0 else x)
    new['floor/max_floor'] = new['floor'] / new['max_floor']
    new['floor/max_floor'] = new['floor/max_floor'].fillna(0)
    new['kitch_sq/full_sq'] = new['kitch_sq'] / new['full_sq']
    new['kitch_sq/full_sq'] = new['kitch_sq/full_sq'].fillna(0)
    new = new.drop('timestamp', axis=1)

    return new

Разделим выборку на обучающую и тестовую еще раз (потому что дополнительные признаки созданы для исходной выборки).

In [ ]:
my_df = new_features(df)
X = my_df.drop('price', axis=1)
y = my_df['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
my_df.isna().sum().sum()

0

### Model Selection

Посмотрии, какого качества можно добиться если использовать разные модели:
* `DecisionTreeRegressor` из `sklearn`
* `RandomForestRegressor` из `sklearn`
* `CatBoostRegressor`

 # Decision Tree Regressor

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
grid_params = {'max_depth': range(4, 20, 1),
               'min_samples_split': range(7, 20, 3),
               'min_samples_leaf': range(7, 20, 3)}
dtr_grid = GridSearchCV(estimator=DecisionTreeRegressor(), param_grid=grid_params, cv=3)
dtr_grid.fit(X_train, y_train)

In [ ]:
dtr_grid.best_estimator_

DecisionTreeRegressor(max_depth=8, min_samples_leaf=19, min_samples_split=7)

In [ ]:
dtr = DecisionTreeRegressor(max_depth=8, min_samples_leaf=19, min_samples_split=7)
dtr.fit(X_train, y_train)
predict_dtr = dtr.predict(X_test)

In [ ]:
mean_squared_error(y_test, predict_dtr, squared=False)

2912110.0035656523

# CatBoost

In [ ]:
!pip install catboost

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostRegressor
from catboost import Pool, cv

In [ ]:
cat = CatBoostRegressor()
cat_params = {'learning_rate': np.arange(0.2, 0.55, 0.05),
        'depth': range(5, 10, 1),
        'l2_leaf_reg': [1, 3]}
grid_result = cat.grid_search(cat_params, X=X_train, y=y_train, plot=True, cv=3)


In [ ]:
from catboost import Pool, cv
params = {"iterations": 100,
          "depth": 7,
          "loss_function": "RMSE",
          "verbose": False,
          "learning_rate": 0.2,
          'l2_leaf_reg': 1}
cv_dataset = Pool(data=X_train,
                  label=y_train)
scores = cv(cv_dataset,
            params,
            fold_count=3,
            plot="True")

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Training on fold [0/3]

bestTest = 2648052.074
bestIteration = 71

Training on fold [1/3]

bestTest = 2803778.148
bestIteration = 77

Training on fold [2/3]

bestTest = 2600340.952
bestIteration = 97



In [ ]:
from sklearn.metrics import mean_squared_error
cat = CatBoostRegressor(depth=7, learning_rate=0.2, l2_leaf_reg=1, iterations=90)
cat.fit(X_train, y_train, verbose=False)
predict_cat = cat.predict(X_test)
mean_squared_error(y_test, predict_cat, squared=False)

# RandomForest

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV


In [ ]:
grid_params = {'n_estimators': range(30, 50, 5),
               'max_depth': range(10, 22, 3)}

In [ ]:
grid = GridSearchCV(estimator=RandomForestRegressor(), param_grid=grid_params, cv=3)

In [ ]:
grid.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=RandomForestRegressor(),
             param_grid={'max_depth': range(10, 22, 3),
                         'n_estimators': range(30, 50, 5)})

In [ ]:
grid.best_estimator_

RandomForestRegressor(max_depth=19, n_estimators=40)

In [ ]:
from sklearn.metrics import mean_squared_error
rfc = RandomForestRegressor(max_depth=25, n_estimators=40)
rfc.fit(X_train, y_train)
predict_rfc = rfc.predict(X_test)
mean_squared_error(y_test, predict_rfc, squared=False)

2476517.2335063405

### Ensemble



In [ ]:
time_train_df = X_train.copy()
time_train_df['price'] = y_train
time_test_df = X_test.copy()
time_test_df['price'] = y_test

In [ ]:
inv_train_df = time_train_df[time_train_df['Investment'] == 1]
own_train_df = time_train_df[time_train_df['Investment'] == 0]
inv_test_df = time_test_df[time_test_df['Investment'] == 1]
own_test_df = time_test_df[time_test_df['Investment'] == 0]
X_inv_train = inv_train_df.drop('price', axis=1)
y_inv_train = inv_train_df['price']
X_own_train = own_train_df.drop('price', axis=1)
y_own_train = own_train_df['price']
X_inv_test = inv_test_df.drop('price', axis=1)
y_inv_test = inv_test_df['price']
X_own_test = own_test_df.drop('price', axis=1)
y_own_test = own_test_df['price']

In [ ]:
cat_inv = CatBoostRegressor()
cat_params = {'learning_rate': np.arange(0.2, 0.31, 0.02),
        'depth': range(3, 6, 1),
        'l2_leaf_reg': [1]}
grid_result = cat_inv.grid_search(cat_params, X=X_own_train, y=y_own_train, plot=True, cv=3)

In [ ]:
cat_own = CatBoostRegressor()
cat_params = {'learning_rate': np.arange(0.2, 0.31, 0.02),
        'depth': range(5, ),
        'l2_leaf_reg': [1]}
grid_result = cat_own.grid_search(cat_params, X=X_own_train, y=y_own_train, plot=True, cv=3)

In [ ]:
model_invest_cat = CatBoostRegressor(depth=4, learning_rate=0.22, l2_leaf_reg=1, iterations=100)
model_owner_cat = CatBoostRegressor(depth=5, learning_rate=0.22, l2_leaf_reg=1, iterations=100)
model_invest_cat.fit(X_inv_train, y_inv_train)
model_owner_cat.fit(X_own_train, y_own_train)
pred_inv_cat = model_invest_cat.predict(X_inv_test)
pred_own_cat = model_owner_cat.predict(X_own_test)

In [ ]:
y_test_full = pd.concat((y_inv_test, y_own_test))
pred_cat = pd.concat((pd.Series(pred_inv_cat), pd.Series(pred_own_cat)))
mean_squared_error(pred_cat, y_test_full, squared=False)

2279619.6285880893

In [ ]:
model_invest_rfc = RandomForestRegressor(max_depth=25, n_estimators=40)
model_owner_rfc = RandomForestRegressor(max_depth=25, n_estimators=40)
model_invest_rfc.fit(X_inv_train, y_inv_train)
model_owner_rfc.fit(X_own_train, y_own_train)
pred_inv_rfc = model_invest_rfc.predict(X_inv_test)
pred_own_rfc = model_owner_rfc.predict(X_own_test)

In [ ]:
pred_rfc = pd.concat((pd.Series(pred_inv_rfc), pd.Series(pred_own_rfc)))
mean_squared_error(pred_rfc, y_test_full, squared=False)

2404475.869837812

### Ensemble v.2

Попробуем сделать для `Investment` более сложную модель: обучим `CatBoostRegressor` и `HuberRegressor` из `sklearn`, а затем сложим их предсказания с весами `w_1` и `w_2` (сумма весов равняется 1).

In [ ]:
from sklearn.linear_model import HuberRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [ ]:
pipe = Pipeline([('scale', StandardScaler()),
                ('model', HuberRegressor())])

In [ ]:
hreg_grid = GridSearchCV(estimator=pipe,
                                   param_grid={'model__alpha': np.arange(0.1, 1, 0.1),
                                              'model__max_iter': [200]}, cv=5)
hreg_grid.fit(X_inv_train, y_inv_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scale', StandardScaler()),
                                       ('model', HuberRegressor())]),
             param_grid={'model__alpha': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]),
                         'model__max_iter': [200]})

In [ ]:
hreg_grid.best_estimator_

Pipeline(steps=[('scale', StandardScaler()),
                ('model', HuberRegressor(alpha=0.9, max_iter=200))])

In [ ]:
cat_invest = CatBoostRegressor(depth=4, learning_rate=0.22, l2_leaf_reg=1, iterations=100)
hreg_invest = HuberRegressor(alpha=0.9, max_iter=200)

In [ ]:
scaler = StandardScaler()

In [ ]:
cat_invest.fit(X_inv_train, y_inv_train)
hreg_invest.fit(scaler.fit_transform(X_inv_train), y_inv_train)

0:	learn: 4456756.3996694	total: 25.3ms	remaining: 2.51s
1:	learn: 4132535.0142035	total: 36.5ms	remaining: 1.79s
2:	learn: 3895016.4411115	total: 46.8ms	remaining: 1.51s
3:	learn: 3718488.3272484	total: 57.5ms	remaining: 1.38s
4:	learn: 3585043.9496839	total: 68.9ms	remaining: 1.31s
5:	learn: 3482803.7536905	total: 79ms	remaining: 1.24s
6:	learn: 3396750.1257881	total: 90.4ms	remaining: 1.2s
7:	learn: 3325578.7033125	total: 101ms	remaining: 1.16s
8:	learn: 3261525.6284291	total: 112ms	remaining: 1.13s
9:	learn: 3216327.1020316	total: 121ms	remaining: 1.09s
10:	learn: 3171184.4273811	total: 132ms	remaining: 1.07s
11:	learn: 3131297.9810085	total: 142ms	remaining: 1.04s
12:	learn: 3093900.8805505	total: 152ms	remaining: 1.02s
13:	learn: 3059619.1748973	total: 163ms	remaining: 998ms
14:	learn: 3040223.3047236	total: 173ms	remaining: 978ms
15:	learn: 3026540.8225273	total: 183ms	remaining: 962ms
16:	learn: 3004149.1530020	total: 194ms	remaining: 947ms
17:	learn: 2991467.3167421	total: 203

HuberRegressor(alpha=0.9, max_iter=200)

In [ ]:
cat_pred_inv_v2 = cat_invest.predict(X_inv_test)
hreg_pred_inv_v2 = hreg_invest.predict(scaler.transform(X_inv_test))

In [ ]:
k = 1 # при переборе получилось так, что минимальный RMSE достигается при использовании только catboost

In [ ]:
pred_inv_v2 = k * cat_pred_inv_v2 + (1 - k) * hreg_pred_inv_v2
pred_v2 = pd.concat((pd.Series(pred_inv_v2), pd.Series(pred_own_cat)))

In [ ]:
mean_squared_error(pred_v2, y_test_full, squared=False)

2260740.682680031

Таким образом, наилучшей моделью оказался ансамбль CatBoostRegressor'ов, обученных отдельно для Investment и OwnerOccupier